In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import datetime
from pymystem3 import Mystem
import re
from nltk.corpus import stopwords

conn = sqlite3.connect(r"../../diplom.db")
cur = conn.cursor()

res = cur.execute("""SELECT ID, DDATE, TRANSCRIPT, DEMOCRACY_COUNT FROM VEDOMOSTI ORDER BY DEMOCRACY_COUNT DESC LIMIT 25;""")
for i, date, text, count in res:
    print(i, date, count)
    with open(f'{i}.txt', 'w', encoding='utf-8') as f:
        f.write(text)

1762802 2010-11-11 21:29:32 33
1767261 2012-08-27 20:47:50 28
1757963 2019-05-23 21:51:14 22
1726347 2012-06-07 22:38:03 19
1746880 2012-02-12 20:25:18 19
1754643 2017-03-30 21:27:52 19
1760950 2010-10-07 20:32:22 19
1745336 2017-11-29 21:03:33 17
1748894 2015-05-17 19:02:24 17
1749442 2017-11-29 21:03:33 17
1721783 2016-01-13 20:11:39 16
1763287 2016-09-01 19:45:58 16
1724553 2016-08-16 21:01:16 14
1727067 2019-11-14 23:19:27 13
1733605 2015-04-14 19:23:03 13
1736519 2019-11-14 23:19:27 13
1756194 2010-02-01 21:16:54 13
1747457 2017-01-11 22:35:17 12
1719438 2015-08-11 20:45:17 11
1725062 2008-07-14 20:00:00 11
1735413 2010-10-19 12:09:47 11
1738853 2011-08-18 21:59:04 11
1745957 2010-09-10 13:41:58 11
1735207 2021-07-01 22:17:19 10
1743300 2021-07-01 22:17:19 10
